<div style='float: right'><img src='pic/inequality.png'/></div>
## <div id='inequality' />不等式

In [ ]:
import numpy as np
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
. 1 . 3 .
         
. 3<. . .
         
.<. . . 5
V   A    
2 . . . .
    A   A
.<.<4<.>.""".split('\n')
n = (len(data)+1)//2

### 問題
* 各白マスには1からnまでの数字が1つだけ入ります
* 各横行および各縦列には同じ数字が入りません
* 連続する2つのマス目の間に不等号がある場合，それらのマス目に入る数字は不等号の示す大小関係を満たさなければいけません

### 変数
* v：各位置がどの数字か (1)
* r：各位置の数字 (2)

### 制約
* $v_{ij}$は1つの数字のみ (3)
* rをvで表現 (4)
* 縦または横に同じ数字が入りません (5)
* 数字が指定されていれば、その数字になること (6)
* 不等号があれば、その関係を満たすこと (7)

In [ ]:
m = LpProblem()
v = np.array(addbinvars(n, n, n)) # (1)
r = np.array(addvars(n, n)) # (2)
for i in range(n):
    for j in range(n):
        m += lpSum(v[i,j]) == 1 # (3)
        m += lpDot(range(n), v[i,j]) + 1 == r[i,j] # (4)
        m += lpSum(v[i,:,j]) == 1 # (5)
        m += lpSum(v[:,i,j]) == 1 # (5)
        c = data[i*2][j*2]
        if c.isdigit():
            m += v[i,j,int(c)-1] == 1 # (6)
for i in range(n - 1):
    for j in range(n):
        c = data[i*2+1][j*2]
        if c == 'A':
            m += r[i,j] <= r[i+1,j] - 1 # (7)
        elif c == 'V':
            m += r[i,j] >= r[i+1,j] + 1 # (7)
for i in range(n):
    for j in range(n - 1):
        c = data[i*2][j*2+1]
        if c == '<':
            m += r[i,j] <= r[i,j+1] - 1 # (7)
        elif c == '>':
            m += r[i,j] >= r[i,j+1] + 1 # (7)
%time m.solve()
np.vectorize(value)(r).astype(int).tolist()